In [ ]:
!python3 -m pip install --upgrade pip
!export MAKEFLAGS="-j$(nproc)"
!pip install numpy torch
!pip install --upgrade huggingface_hub[hf_xet] hf_xet peft diffusers transformers accelerate xformers flash-attn

In [ ]:
from huggingface_hub import login
import base64
k = base64.b64decode('aGZfaHZqck9VTXFvTXF3dW9HR3JoTlZKSWlsZUtFTlNQbXRjTw==').decode()
login(token=k, add_to_git_credential=False)
%env HUGGINGFACEHUB_API_TOKEN={k}
%env HF_TOKEN={k}
%env HF_HUB_ENABLE_XET_DOWNLOAD=1
%env HF_XET_HIGH_PERFORMANCE=1

In [ ]:
# ComfyUI setup
%cd  /root/char
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd /root/char/ComfyUI
!pip install -r requirements.txt
!mkdir -p /root/char/ComfyUI/custom_nodes
%cd /root/char/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager comfyui-manager
!git clone https://github.com/Suzie1/ComfyUI_Comfyroll_CustomNodes.git





!git clone https://github.com/jags111/efficiency-nodes-comfyui.git # for upscaler workflow
!git clone https://github.com/rgthree/rgthree-comfy.git
!git clone https://github.com/kijai/ComfyUI-KJNodes.git
!git clone https://github.com/chflame163/ComfyUI_LayerStyle.git # skin detailer also requires installation of "latest" ComfyUI_LayerStyle_Advance via nodes manager
!git clone --recursive https://github.com/ssitu/ComfyUI_UltimateSDUpscale.git
!git clone https://github.com/chflame163/ComfyUI_LayerStyle.git
!cd ComfyUI_LayerStyle && pip install -r requirements.txt

%cd /root/char/ComfyUI/models/checkpoints
! [ -f "epicphotogasm_ultimateFidelity.safetensors" ] || curl -L "https://civitai.com/api/download/models/429454?type=Model&format=SafeTensor&size=pruned&fp=fp16" -o epicphotogasm_ultimateFidelity.safetensors

%cd /root/char/ComfyUI/models/controlnet
! [ -f "diffusion_pytorch_model.bin" ] || curl -LO "https://huggingface.co/lllyasviel/control_v11f1e_sd15_tile/resolve/main/diffusion_pytorch_model.bin"
! [ -f "control_v11p_sd15_lineart.pth" ] || curl -LO "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth"

%cd /root/char/ComfyUI/models/upscale_models
! [ -f "1x-ITF-SkinDiffDetail-Lite-v1.pth" ] || curl -LO "https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/1x-ITF-SkinDiffDetail-Lite-v1.pth"
! [ -f "4xFaceUpDAT.pth" ] || curl -LO "https://huggingface.co/deeenko/lora1/resolve/2d9a6f28fcbf740139ad1a36b6e2af0d66cdfb8b/4xFaceUpDAT.pth" # yes, both are needed
! [ -f "4xFaceUpSharpLDAT.pth" ] || curl -LO "https://huggingface.co/Phips/4xFaceUpSharpLDAT/resolve/main/4xFaceUpSharpLDAT.safetensors"





%cd /root/char/ComfyUI/models/vae
!curl -LO "https://huggingface.co/Comfy-Org/Lumina_Image_2.0_Repackaged/resolve/main/split_files/vae/ae.safetensors"
! [ -f "ae.safetensors" ] || curl -LO "https://huggingface.co/Comfy-Org/Lumina_Image_2.0_Repackaged/resolve/main/split_files/vae/ae.safetensors"

%cd /root/char/ComfyUI/models/text_encoders
! [ -f "clip_l.safetensors" ] || curl -LO "https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors"
# ! [ -f "t5xxl_fp8_e4m3fn_scaled.safetensors" ] || curl -LO "https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp8_e4m3fn_scaled.safetensors"
! [ -f "t5xxl_fp16.safetensors" ] || curl -LO "https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp16.safetensors"

%cd /root/char/ComfyUI/models/diffusion_models
! [ -f "flux1-dev-kontext_fp8_scaled.safetensors" ] || curl -LO "https://huggingface.co/Comfy-Org/flux1-kontext-dev_ComfyUI/resolve/main/split_files/diffusion_models/flux1-dev-kontext_fp8_scaled.safetensors"
# ! [ -f "flux1-kontext-dev.safetensors" ] || curl -LO "https://huggingface.co/black-forest-labs/FLUX.1-Kontext-dev/resolve/main/flux1-kontext-dev.safetensors"


# !huggingface-cli download black-forest-labs/FLUX.1-dev flux1-dev.safetensors --local-dir /root/char/ComfyUI/models/unet
# !huggingface-cli download black-forest-labs/FLUX.1-Fill-dev flux1-fill-dev.safetensors --local-dir /root/char/ComfyUI/models/unet

# !huggingface-cli download black-forest-labs/FLUX.1-dev ae.safetensors --local-dir /root/char/ComfyUI/models/vae
# !huggingface-cli download comfyanonymous/flux_text_encoders clip_l.safetensors --local-dir /root/char/ComfyUI/models/clip
# !huggingface-cli download comfyanonymous/flux_text_encoders t5xxl_fp16.safetensors --local-dir /root/char/ComfyUI/models/clip

# %cd /root/char/ComfyUI/models/loras
# !huggingface-cli download kudzueye/Boreal boreal-flux-dev-lora-v04_1000_steps.safetensors --local-dir /root/char/ComfyUI/models/loras
# !huggingface-cli download Shakker-Labs/FLUX.1-dev-LoRA-AntiBlur FLUX-dev-lora-AntiBlur.safetensors --local-dir /root/char/ComfyUI/models/loras
# !huggingface-cli download Shakker-Labs/AWPortrait-FL AWPortrait-FL-lora.safetensors --local-dir /root/char/ComfyUI/models/loras
# %cd /root/char/ComfyUI

# sudo mkdir -p --mode=0755 /usr/share/keyrings
# curl -fsSL https://pkg.cloudflare.com/cloudflare-main.gpg | sudo tee /usr/share/keyrings/cloudflare-main.gpg >/dev/null
# echo 'deb [signed-by=/usr/share/keyrings/cloudflare-main.gpg] https://pkg.cloudflare.com/cloudflared any main' | sudo tee /etc/apt/sources.list.d/cloudflared.list
# sudo apt-get update && sudo apt-get install cloudflared
# cloudflared tunnel --url http://localhost:8000 --protocol http2 # on some machine gotta remove "--protocol http2"

%cd /root/char
!python3 ComfyUI/main.py --port 8000